# Polymer Partitioning in Two Fluid System
---
This exercise is to demonstrate a simple workflow utilizing the entire set of MoSDeF tools to write out a simulation file to use in LAMMPS. The main feature of this workflow is to demonstrate methods to use GMSO topologies instead of a ParmEd/OPENMM backend. This rerouted workflow, through a GMSO topology, gives the users the increased functionality and testing supported through GMSO. Although the workflow process remains relatively similar, small syntax differences allow users to gain increased functionality demonstrated in this workflow, such as using **forcefield matching** to individual molecules in the system, and to write **custom potential forms**. </br>

#### Note this workflow uses features that are still under development on the GMSO side of things. 
As such, it is necessary to install dev version of specific branches to access this functionality.
1. Install mBuild 
```bash
git clone https://github.com/mosdef-hub/mbuild.git
pip install -e ./
2. Install GMSO
```bash
git clone https://github.com/daico007/gmso.git
git fetch flatten_mbuild_convert
git checkout flatten_mbuild_convert
pip install ./
```
3. Install forcefield-utilities
```bash
conda install forcefield-utilities -c conda-forge
```
___


### Exercise Stages:
1. Import libraries
2. Custom mBuild recipes
3. Build partitioning box
4. Parameterize with multiple forcefields
5. Run Cassandra Simulations
6. Evaluate Results
---

# 1. Import Libraries
---

In [1]:
# Import Libraries
import numpy as np
import mbuild as mb
import forcefield_utilities as ffutils
import gmso


# 2. Custom mBuild Recipes
---

In [2]:
from scipy.constants import N_A
def Packing_Number(compound, vol):
    """
    Identify the number of compounds to place into a box.
    
    NOTES
    -----
    The compound must have the attribute `compound.dens` which is used to
    calculate the number of compounds to fit into the volume.
    """
    n_compounds = compound.dens * vol / compound.mass * N_A * 1e-21
    return int(n_compounds)

In [3]:
import operator
import functools
def Partitioned_Box(solute, solvent1, solvent2, boxl, frac_interface=0):
    """
    Solubilize the `solute` into `solvent1` and fill other half of the 
    box, made as a cubic box with sidelengths `boxl` with `solvent2`.
    """
    # Pack mbuild box with water, polymer, and hexane
    full_box = mb.Box([boxl, boxl, boxl])
    half_box = mb.Box([boxl/2, boxl, boxl])
    vol=functools.reduce(operator.mul, full_box.lengths, 1)/2
    solute.translate(np.array([boxl+boxl*frac_interface, boxl, boxl])/2)

    filled_box1 = mb.packing.solvate(
        solvent=solvent1, 
        solute=solute, 
        box=half_box,  
        n_solvent=Packing_Number(solvent1, vol),
        edge=0.01
    )
    filled_box1.name = "sol1"
    filled_box2 = mb.packing.fill_box(
        compound=solvent2,
        box=half_box,  
        n_compounds=Packing_Number(solvent2, vol),
        edge=0.01
    )
    filled_box2.translate([boxl/2,0,0])
    filled_box2.name = "sol2"
    partitioned_box = mb.Compound()
    partitioned_box.add(filled_box1)
    partitioned_box.add(filled_box2)
    return partitioned_box

# 3. Build Box of Molecules
---

In [4]:
# Build Polymer
monomer = mb.load("CCO", smiles=True)
monomer.name = "monomer"
polymer= mb.lib.recipes.Polymer()
polymer.add_monomer(monomer, indices=(3,7))
polymer.build(n=10)
polymer.name = "polymer"

# Build Solvent 1
water = mb.load("O", smiles=True)
water.name = 'water'
water.dens = 0.99

# Build Solvent 2
cyclopentane = mb.load("C1CCCC1", smiles=True)
cyclopentane.name = "cyclopentane"
cyclopentane.dens = 0.63

# Build Partitioned Box
boxl = 2 # Use a cubic boxlength of 3 nm
solute_position = 0 # center the polymer into the center of the box
partitioned_box = Partitioned_Box(polymer, water, cyclopentane, boxl, solute_position)
#partitioned_box = partitioned_box.group_by_molecules()
partitioned_box.visualize()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/mbuild/mbuild/compound.py:1251: UserWarning: Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box.
  "Compound.box.lengths < Compound.boundingbox.lengths. "
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/mbuild/mbuild/compound.py:773: UserWarning: After adding new Compound, Compound.box.lengths < Compound.boundingbox.lengths. There may be particles outside of the defined simulation box
  "After adding new Compound, Compound.box.lengths < "


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# 4. Parameterize with Multiple Forcefields
---

## Load forcefields

In [5]:
import forcefield_utilities as ffutils
ffloader = ffutils.FoyerFFs() # ffloader is now an object where we can load in a forcefield for repeated uses. 
# In order to use a gmsoFF, we convert this Foyer forcefield to a GMSO forcefield.
alcohols_ff = ffloader.load("xmls/alcohols.xml").to_gmso_ff()
solvent_ff = ffloader.load("xmls/alkanes.xml").to_gmso_ff()
water_ff = ffloader.load("xmls/tip3p.xml").to_gmso_ff() # 3 different foyer xmls located locally
#oplsaa = ffloader.load("../../../switchable_interfaces_fall/22-03-27-full_screening/utils/Forcefields/oplsaa_switchable.xml").to_gmso_ff()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/atom_type.py:180: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gomc-gmso-paper/gmso/gmso/core/atom_type.py:192: UserWarning: Charges are assumed to be in units of elementary charge
  UNIT_WARNING_STRING.format("Charges", "elementary charge")


## Convert Topology to GMSO

from gmso.external import from_mbuild
import time
start = time.time()
topology_gmso = from_mbuild(partitioned_box) # Create GMSO topology
print("Time to convert mbuild structure: ", time.time()-start)
topology_gmso.identify_connections() # Identify angles and dihedrals (this may be slow, 
# changes to speed will come soon

## Apply forcefields - by molecule info

In [6]:
# These are experimental features, and subject to slight changes in the API
import warnings
warnings.simplefilter("ignore", UserWarning)
from gmso.external import from_mbuild
import time
start = time.time()
topology_gmso = from_mbuild(partitioned_box) # Create GMSO topology
print("Time to convert mbuild structure: ", time.time()-start)
start = time.time()
topology_gmso.identify_connections() # Identify angles and dihedrals (this may be slow, 
print("Time to id connections: ", time.time()-start)
from gmso.parameterization import apply
ff_dicts = {
    "water": water_ff,
    "cyclopentane": solvent_ff,
    "polymer": alcohols_ff
} #The names here are from the molecule names that were put into the box, and can be found
#by looking at topology_gmso.subtops
#ff_dicts2 = {"sol1": oplsaa, "sol2": oplsaa}
start = time.time()
apply(topology_gmso, ff_dicts, identify_connected_components=False, match_ff_by="molecule",
                  use_molecule_info=True) # apply forcefield to relevant subtops
print("Time to apply forcefields: ", time.time()-start)
from gmso.core.views import PotentialFilters
print(len(topology_gmso.atom_types(PotentialFilters.REPEAT_DUPLICATES)))
#assert topology_gmso.is_typed()

Time to convert mbuild structure:  0.22959089279174805
Time to id connections:  1.7715110778808594
Time to apply forcefields:  0.8011651039123535
783


In [8]:
# These are experimental features, and subject to slight changes in the API
import warnings
warnings.simplefilter("ignore", UserWarning)
from gmso.external import from_mbuild
import time
start = time.time()
topology_gmso = from_mbuild(partitioned_box) # Create GMSO topology
print("Time to convert mbuild structure: ", time.time()-start)
start = time.time()
topology_gmso.identify_connections() # Identify angles and dihedrals (this may be slow, 
print("Time to id connections: ", time.time()-start)
from gmso.parameterization import apply
ff_dicts = {
    "water": water_ff,
    "cyclopentane": solvent_ff,
    "polymer": alcohols_ff
} #The names here are from the molecule names that were put into the box, and can be found
#by looking at topology_gmso.subtops
start = time.time()
apply(topology_gmso, ff_dicts, identify_connected_components=False, match_ff_by="molecule",
                  use_molecule_info=True) # apply forcefield to relevant subtops
print("Time to apply forcefields: ", time.time()-start)
from gmso.core.views import PotentialFilters
print(len(topology_gmso.atom_types(PotentialFilters.REPEAT_DUPLICATES)))
assert topology_gmso.is_typed()

Time to convert mbuild structure:  0.7166788578033447
Time to id connections:  4.084200382232666
Time to apply forcefields:  80.4165849685669
1476


In [14]:
for site in topology_gmso.sites[0:1000:100]:
    print(site.molecule, site.residue, site.group)
    print(site.atom_type)

Molecule(name='polymer', number=0) Residue(name='monomer', number=5) sol1
None
Molecule(name='water', number=9) Residue(name='water', number=9) sol1
None
Molecule(name='water', number=42) Residue(name='water', number=42) sol1
None
Molecule(name='water', number=76) Residue(name='water', number=76) sol1
None
Molecule(name='water', number=109) Residue(name='water', number=109) sol1
None
Molecule(name='water', number=142) Residue(name='water', number=142) sol1
None
Molecule(name='water', number=176) Residue(name='water', number=176) sol1
None
Molecule(name='water', number=209) Residue(name='water', number=209) sol1
None
Molecule(name='water', number=242) Residue(name='water', number=242) sol1
None
Molecule(name='cyclopentane', number=3) Residue(name='cyclopentane', number=3) sol2
None


## Apply forcefields using isomorphs

In [9]:
from gmso.external import from_mbuild
import time
start = time.time()
topology_gmso = from_mbuild(partitioned_box) # Create GMSO topology
print("Time to convert mbuild structure: ", time.time()-start)
start = time.time()
topology_gmso.identify_connections() # Identify angles and dihedrals (this may be slow, 
print("Time to id connections: ", time.time()-start)
from gmso.parameterization import apply
import warnings
warnings.simplefilter("ignore", UserWarning)
ff_dicts = {
    "water": water_ff,
    "cyclopentane": solvent_ff,
    "polymer": alcohols_ff
} #The names here are from the molecule names that were put into the box, and can be found
#by looking at topology_gmso.subtops
start = time.time()
apply(topology_gmso, ff_dicts, identify_connected_components=True,
                  use_molecule_info=False) # apply forcefield to relevant subtops
print("Time to apply forcefields: ", time.time()-start)
print(len(topology_gmso.atom_types))
assert topology_gmso.is_fully_typed

Time to convert mbuild structure:  16.46244788169861
Time to id connections:  103.92052221298218
Time to apply forcefields:  3.2944939136505127
11


In [18]:
from gmso.core.views import PotentialFilters
print(len(topology_gmso.atom_types(PotentialFilters.REPEAT_DUPLICATES)))

11346


# 5. Write out LAMMPS Configuration File
---

In [11]:
#topology_gmso.save("config.lammps", overwrite=True) # Write out lammps file
# This may also be slow (~3 minutes), the writers are being revamped to speed up this process

In [12]:
#!head -n 10 config.lammps